In [25]:
import numpy as np
import pandas as pd
from gurobipy import *

In [26]:
df = pd.read_excel('real_distances_10customers.xls') # dataframe
nodes = range(df.shape[0])# List of nodes

In [27]:
dist = [[df[j][i] for j in nodes] for i in nodes]# Build distance matrix

defining demand as unit demand for each nodes

In [29]:
demand = np.ones(len(nodes))

the minimum number of vehicles is 2 since summation of demand/capacity = 2

In [31]:
k = 2 # number of vehicles

In [32]:
veh_cap = 5 # vehicle capacity

In [33]:
#Create a new model
vrp = Model("vrp_model")

In [34]:
#Create variables
x = vrp.addVars(nodes, nodes, lb = 0,vtype = GRB.BINARY,name = 'x')
u = vrp.addVars(nodes, lb = 0,vtype = GRB.INTEGER,name = 'u')

In [35]:
#Add constraint
vrp.addConstrs(((quicksum(x[i,j] for j in nodes if j != i) == 1) for i in nodes if i != 0), name = '1');
vrp.addConstrs(((quicksum(x[i,j] for i in nodes if i != j) == 1) for j in nodes if j != 0), name = '2');
vrp.addConstr(((quicksum(x[0,j] for j in nodes) == k)), name = '3');
vrp.addConstr(((quicksum(x[i,0] for i in nodes) == k)), name = '4');
vrp.addConstrs(((u[i] - u[j] + veh_cap*x[i,j] <= veh_cap - demand[j]) for i in nodes for j in nodes if i!=j and i != 0 and j != 0), name = '5');
vrp.addConstrs(((u[i] <= veh_cap) for i in nodes), name = '6');
vrp.addConstrs(((u[i] >= demand[i]) for i in nodes), name = '7');

In [36]:
#set objective function
vrp.setObjective((quicksum(dist[i][j]*x[i,j] for i in nodes for j in nodes if i != j)),GRB.MINIMIZE)

In [37]:
vrp.setParam("TimeLimit", 600)   # time limit
#vrp.setParam("OutputFlag", 0)     # do not print on console

Set parameter TimeLimit to value 600


In [38]:
vrp.update()
vrp.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 134 rows, 132 columns and 514 nonzeros
Model fingerprint: 0x1847306b
Variable types: 0 continuous, 132 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [4e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 22 rows and 11 columns
Presolve time: 0.01s
Presolved: 112 rows, 121 columns, 492 nonzeros
Variable types: 0 continuous, 121 integer (111 binary)
Found heuristic solution: objective 2052.0000000

Root relaxation: objective 1.059493e+03, 38 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [39]:
status = vrp.status

object_Value = vrp.objVal

print()
print("model status is: ", status)
print()
print("Objective value is: ", object_Value)


model status is:  2

Objective value is:  1721.1


In [40]:
# Print decision varibales which are not zeros
if status !=3 and status != 4:
    for v in vrp.getVars(): 
        if vrp.objVal < 1e+99  and v.x!=0:
            print('%s %f'%(v.Varname,v.x))

x[0,3] 1.000000
x[0,4] 1.000000
x[1,9] 1.000000
x[2,1] 1.000000
x[3,8] 1.000000
x[4,10] 1.000000
x[5,0] 1.000000
x[6,7] 1.000000
x[7,5] 1.000000
x[8,6] 1.000000
x[9,0] 1.000000
x[10,2] 1.000000
u[0] 1.000000
u[1] 4.000000
u[2] 3.000000
u[3] 1.000000
u[4] 1.000000
u[5] 5.000000
u[6] 3.000000
u[7] 4.000000
u[8] 2.000000
u[9] 5.000000
u[10] 2.000000


In [41]:
# printing routes    
if status != 3 and status != 4:
    vis = []
    Sol_x = np.zeros([len(nodes), len(nodes)])
    for i in nodes:
        for j in nodes:
            if vrp.objVal < 1e+99:
                Sol_x[i,j] = x[i,j].getAttr("X")
            else:
                error_status = True
                ofvv = 1e+99
            if 1 - 0.00001 <= Sol_x[i,j] <= 1 + 0.00001:
                vis.append((i,j))

visited = np.array(vis)
solution = []
if visited[0][0] == 0:
    sol = [visited[0][0], visited[0][1]]
elif visited[0][0] !=0 and visited[0][1] == 0:
    sol = [visited[0][1], visited[0][0]] 
else:
    print('First tuple should include depot 0')
visited = np.delete(visited,0, axis = 0)

solution = []
for i in visited:
    try:
        next_ind = int(np.where(visited[:,0] == sol[-1])[0])
        sol.append(visited[next_ind][1])
        visited = np.delete(visited,next_ind, axis = 0)
    except:
        next_ind = int(np.where(visited[:,1] == sol[-1])[0])
        sol.append(visited[next_ind][0])
        visited = np.delete(visited,next_ind, axis = 0)
    
    if sol[0] == sol[-1]:
        sol = np.asarray(sol)
        solution.append(sol)
        used = []
        for j in solution:
            for k in j:
                used.append(k)
        remain = list(set(nodes) - set(used))
        if remain == []:
            break
        sol=[visited[0][0], visited[0][1]]
        visited = np.delete(visited,0, axis = 0)

print('optimal Solution is:', solution)                    

optimal Solution is: [array([0, 3, 8, 6, 7, 5, 0]), array([ 0,  4, 10,  2,  1,  9,  0])]


C:\Users\murat\AppData\Local\Temp\ipykernel_16220\2157032967.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_ind = int(np.where(visited[:,0] == sol[-1])[0])
